In [1]:
import csv
import numpy as np
import copy
import time
import math

In [2]:
def split_train_label(data):
    train_x = []
    train_y = []
    for i in data:
        train_x.append(i[1:])
        train_y.append([i[0]])
        
    return train_x,train_y

In [3]:
with open('titanic_data.csv','r') as file:
    temp = csv.reader(file)
    data = list(temp)

header = data[0]
data = data[1:]
for i in range(len(data)):
    row_len = len(data[0])
    for j in range(row_len):
        data[i][j] = float(data[i][j])
    
train_x, train_y = split_train_label(data)

In [4]:
header

['Survived',
 'Pclass',
 'Sex',
 'Age',
 'Siblings/Spouses Aboard',
 'Parents/Children Aboard',
 'Fare']

In [5]:
#normalization
for i in range(len(header[1:])):
    total = 0
    avg = 0
    if header[1:][i] not in ['Sex','Pclass']:
        for j in train_x:
            total += j[i]  
        avg = total/len(train_x)      
        for j in range(len(train_x)):
            train_x[j][i] = train_x[j][i]/avg

In [6]:
train_x

[[3.0, 0.0, 0.7464853332884488, 1.9034334763948497, 0.0, 0.22442054489674754],
 [1.0, 1.0, 1.2893837574982299, 1.9034334763948497, 0.0, 2.2065430383501137],
 [3.0, 1.0, 0.8822099393408941, 0.0, 0.0, 0.24531487149058265],
 [1.0, 1.0, 1.187590302958896, 1.9034334763948497, 0.0, 1.6436870253816958],
 [3.0, 0.0, 1.187590302958896, 0.0, 0.0, 0.2491841912301818],
 [3.0, 0.0, 0.9161410908540054, 0.0, 0.0, 0.26182293722760824],
 [1.0, 0.0, 1.8322821817080108, 0.0, 0.0, 1.6053807599596648],
 [3.0,
  0.0,
  0.06786230302622262,
  5.710300429184549,
  2.6088235294117643,
  0.6523673080964075],
 [3.0, 1.0, 0.9161410908540054, 0.0, 5.217647058823529, 0.3446263796550289],
 [2.0, 1.0, 0.47503612118355837, 1.9034334763948497, 0.0, 0.9308283202042918],
 [3.0,
  1.0,
  0.13572460605244524,
  1.9034334763948497,
  2.6088235294117643,
  0.5169411172104391],
 [1.0, 1.0, 1.9680067877604561, 0.0, 0.0, 0.8218435126908479],
 [3.0, 0.0, 0.6786230302622263, 0.0, 0.0, 0.2491841912301818],
 [3.0,
  0.0,
  1.323314

In [7]:
## encoding class

total_data = []
for i in train_x:
    temp = []
    if i[0] == 1:
        temp.append(1)
    else:
        temp.append(0)
    if i[0] == 2:
        temp.append(1)
    else:
        temp.append(0)
    if i[0] == 3:
        temp.append(1)
    else:
        temp.append(0)
    temp = temp + i[1:]
    total_data.append(temp)
train_x = total_data

In [8]:
## encoding class

train_x

[[0,
  0,
  1,
  0.0,
  0.7464853332884488,
  1.9034334763948497,
  0.0,
  0.22442054489674754],
 [1,
  0,
  0,
  1.0,
  1.2893837574982299,
  1.9034334763948497,
  0.0,
  2.2065430383501137],
 [0, 0, 1, 1.0, 0.8822099393408941, 0.0, 0.0, 0.24531487149058265],
 [1,
  0,
  0,
  1.0,
  1.187590302958896,
  1.9034334763948497,
  0.0,
  1.6436870253816958],
 [0, 0, 1, 0.0, 1.187590302958896, 0.0, 0.0, 0.2491841912301818],
 [0, 0, 1, 0.0, 0.9161410908540054, 0.0, 0.0, 0.26182293722760824],
 [1, 0, 0, 0.0, 1.8322821817080108, 0.0, 0.0, 1.6053807599596648],
 [0,
  0,
  1,
  0.0,
  0.06786230302622262,
  5.710300429184549,
  2.6088235294117643,
  0.6523673080964075],
 [0,
  0,
  1,
  1.0,
  0.9161410908540054,
  0.0,
  5.217647058823529,
  0.3446263796550289],
 [0,
  1,
  0,
  1.0,
  0.47503612118355837,
  1.9034334763948497,
  0.0,
  0.9308283202042918],
 [0,
  0,
  1,
  1.0,
  0.13572460605244524,
  1.9034334763948497,
  2.6088235294117643,
  0.5169411172104391],
 [1, 0, 0, 1.0, 1.9680067877

In [66]:
# each x data is a row vector. y is a column vector. Need header to do normalization

class logistic_titanic:
    def __init__(self, gradientRate= 3/4, max_iter = 1000, abstol = 1e-3, add_intercept = True):
        self.max_iter =  max_iter
        self.abstol = abstol
        #self.reltol = reltol
        self.add_intercept = add_intercept
        self.gradientRate = gradientRate
        self.likelihoodScore = None
        
    def likelihood_score(self):
        likelihood = 0
        for i in range(len(self.training_y)):
            temp = 0
            x = np.array([self.training_x[i]]).T
            temp += self.training_y[i]*math.log(1/(1+ (math.exp((-np.dot(self.theta.T,x))))))
            temp += (1-self.training_y[i])*math.log((1/(1+ (math.exp((np.dot(self.theta.T,x)))))))
            likelihood += temp
        self.likelihoodScore = likelihood
        return likelihood
    def gradient(self):
        gradient = np.zeros((len(self.training_x[0]),1))
        for i in range(len(self.training_y)):
            x = np.array([self.training_x[i]]).T
            temp = self.training_y[i] - (1/(1+ (math.exp((-np.dot(self.theta.T,x))))))
            gradient = gradient + temp*x
        
        return gradient
    
    def logistic_predict(self):
        answer = []
        for i in range(len(self.training_y)):
            temp = 0
            temp_x = np.array([self.training_x[i]]).T
            temp = 1/(1+math.exp(-np.dot(self.theta.T,temp_x)))
            if temp > 1/2:
                answer.append(1)
            else:
                answer.append(0)
        return answer
    def hessian(self):
        hessian = []
        temp = [0] * len(self.training_x[0])
        for i in range(len(self.training_x[0])):
            hessian.append(temp)
            
        hessian = np.array(hessian)
        for i in range(len(self.training_y)):
            temp = 0
            x = np.array([self.training_x[i]]).T
            temp = math.exp((-np.dot(self.theta.T,x)))/((1+math.exp((-np.dot(self.theta.T,x))))**2)
            temp = temp * np.dot(x,x.T)
            hessian = hessian + temp
        
        return hessian
    def fit(self,x,y): 
        ## deep copy data
        self.training_x = np.array(copy.deepcopy(x))
        self.training_y = np.array(copy.deepcopy(y))
        
        ## add intercept
        data_num = len(self.training_x)
        if self. add_intercept == True:
            temp = []
            for i in range(data_num):
                temp.append([1])
            self.training_x = np.append(self.training_x,temp,axis = 1)

        ## initialize theta      
        theta = []
        
        for i in range(len(self.training_x[0])):
            theta.append([1])
        
        self.theta = np.array(theta)
            
        
        ## start training
        last_likelihood = float('-inf') 
        parameter_rate = 1/20
        for i in range(self.max_iter):
            if i % 50 == 0:
                parameter_rate = parameter_rate*self.gradientRate
            current_likelihood = self.likelihood_score()
            if abs(current_likelihood - last_likelihood) <= self.abstol:
                break
            last_likelihood = current_likelihood
            gradient_val = self.gradient()
            self.theta = self.theta + parameter_rate*gradient_val
            
            

        return(self.theta)

In [67]:
a =   logistic_titanic()

In [68]:
start = time.time()
theta = a.fit(train_x, train_y)
end = time.time()
print('time = ', end - start)

time =  18.85419511795044


In [73]:
current_hessian = a.hessian() 

fisher = np.linalg.inv(current_hessian)/len(train_x)

In [59]:
round(0.2222,2)

0.22

In [74]:
for i in range(len(unround_hessian)):
    for j in range(len(unround_hessian)):
        unround_hessian[i][j] = format(unround_hessian[i][j], ".2f")

In [75]:
print(fisher)

[[ 1.22051416e+10  1.22051416e+10  1.22051416e+10 -4.44930791e-06
  -1.44864551e-04 -1.06086203e-05  8.97691961e-07 -1.43621015e-05
  -1.22051416e+10]
 [ 1.22051416e+10  1.22051416e+10  1.22051416e+10 -1.55204483e-05
  -1.17915114e-04 -9.54766623e-06 -5.10799684e-07 -2.53167065e-06
  -1.22051416e+10]
 [ 1.22051416e+10  1.22051416e+10  1.22051416e+10 -2.17357854e-05
  -1.06585162e-04 -9.67655103e-06 -7.51375495e-07 -2.58029852e-07
  -1.22051416e+10]
 [-5.68619406e-06 -1.67573344e-05 -2.29726716e-05  4.59177223e-05
  -7.08245260e-06 -2.35996616e-06 -2.01564456e-06 -4.16479576e-07
   1.02967288e-05]
 [-1.47441131e-04 -1.20491693e-04 -1.09161742e-04 -7.08245260e-06
   6.09370981e-05  3.96723477e-06  3.89623267e-07  1.36233277e-06
   5.92298739e-05]
 [-9.68498535e-06 -8.62403132e-06 -8.75291613e-06 -2.35996616e-06
   3.96723477e-06  3.88650985e-06 -7.85593406e-07 -5.88115590e-07
   3.80054581e-06]
 [-5.20618251e-07 -1.92910990e-06 -2.16968571e-06 -2.01564456e-06
   3.89623267e-07 -7.8559340

In [60]:
len(unround_hessian)

9